In [3]:
# 硅基流动API测试
import requests
import json
import time
import sys

class OllamaTester:
    def __init__(self, config):
        self.base_url = config.get("base_url", "http://localhost:11434/v1")
        self.model = config.get("model", "llama3.1:latest").strip()
        self.api_key = config.get("api_key", "ollama")
        self.verify_ssl = config.get("verify_ssl", False)
        self.max_retries = config.get("max_retries", 3)
        
        self.headers = {
            "Content-Type": "application/json",
        }
        if self.api_key:
            self.headers["Authorization"] = f"Bearer {self.api_key}"
    
    def check_ollama_status(self):
        """检查Ollama服务是否运行"""
        try:
            # 尝试访问Ollama原生API端点
            response = requests.get(
                "http://localhost:11434/api/tags",
                timeout=5,
                verify=self.verify_ssl
            )
            if response.status_code == 200:
                models = response.json().get("models", [])
                print("✅ Ollama服务运行正常")
                print(f"   可用模型: {', '.join([m.get('name', '') for m in models])}")
                return True
            return False
        except Exception as e:
            print(f"❌ 无法连接到Ollama服务: {str(e)}")
            print("   请确保Ollama已启动: ollama serve")
            return False
    
    def test_openai_compatible_api(self):
        """测试OpenAI兼容API"""
        print(f"\n🔍 测试OpenAI兼容端点: {self.base_url}")
        
        # 测试列出模型
        try:
            response = requests.get(
                f"{self.base_url}/models",
                headers=self.headers,
                timeout=10,
                verify=self.verify_ssl
            )
            
            if response.status_code == 200:
                models = response.json().get("data", [])
                print(f"✅ OpenAI兼容API可用")
                print(f"   兼容端点检测到模型数量: {len(models)}")
                
                # 检查目标模型是否可用
                target_model_found = any(
                    self.model in model.get("id", "") for model in models
                )
                if target_model_found:
                    print(f"   ✅ 目标模型 '{self.model}' 可用")
                else:
                    print(f"   ⚠️  目标模型 '{self.model}' 未在列表中，可能名称不匹配")
                return True
            else:
                print(f"❌ OpenAI兼容API返回错误: {response.status_code}")
                print(f"   响应: {response.text}")
                return False
                
        except Exception as e:
            print(f"❌ OpenAI兼容API连接失败: {str(e)}")
            return False
    
    def test_chat_completion(self, prompt="Hello, how are you?", max_tokens=100):
        """测试聊天补全功能"""
        print(f"\n💬 测试聊天补全 (模型: {self.model})")
        
        messages = [
            {"role": "user", "content": prompt}
        ]
        
        data = {
            "model": self.model,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": 0.7,
            "stream": False
        }
        
        for attempt in range(self.max_retries):
            try:
                print(f"   尝试 {attempt + 1}/{self.max_retries}...")
                start_time = time.time()
                
                response = requests.post(
                    f"{self.base_url}/chat/completions",
                    headers=self.headers,
                    json=data,
                    timeout=60,
                    verify=self.verify_ssl
                )
                
                elapsed_time = time.time() - start_time
                
                if response.status_code == 200:
                    result = response.json()
                    print(f"✅ 请求成功! 耗时: {elapsed_time:.2f}秒")
                    
                    # 提取回复内容
                    if "choices" in result and len(result["choices"]) > 0:
                        reply = result["choices"][0].get("message", {}).get("content", "")
                        print(f"\n📝 模型回复:")
                        print("-" * 40)
                        print(reply[:500] + ("..." if len(reply) > 500 else ""))
                        print("-" * 40)
                    
                    # 显示token使用情况
                    if "usage" in result:
                        usage = result["usage"]
                        print(f"\n📊 Token使用:")
                        print(f"   输入token: {usage.get('prompt_tokens', 'N/A')}")
                        print(f"   输出token: {usage.get('completion_tokens', 'N/A')}")
                        print(f"   总token: {usage.get('total_tokens', 'N/A')}")
                    
                    return True
                    
                elif response.status_code == 404:
                    print(f"❌ 模型 '{self.model}' 不存在")
                    print("   请使用 'ollama pull' 下载模型")
                    return False
                else:
                    print(f"❌ 请求失败: {response.status_code}")
                    print(f"   响应: {response.text[:200]}")
                    
            except requests.exceptions.Timeout:
                print(f"⏰ 请求超时，尝试重试...")
                time.sleep(1)
            except Exception as e:
                print(f"❌ 请求异常: {str(e)}")
                time.sleep(1)
        
        print(f"⚠️  所有 {self.max_retries} 次尝试都失败")
        return False
    
    def test_streaming(self, prompt="Tell me a short joke", max_tokens=100):
        """测试流式输出"""
        print(f"\n🌊 测试流式输出")
        
        messages = [
            {"role": "user", "content": prompt}
        ]
        
        data = {
            "model": self.model,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": 0.7,
            "stream": True
        }
        
        try:
            print("   开始接收流式响应...")
            print("-" * 40)
            
            response = requests.post(
                f"{self.base_url}/chat/completions",
                headers=self.headers,
                json=data,
                stream=True,
                timeout=60,
                verify=self.verify_ssl
            )
            
            full_response = ""
            if response.status_code == 200:
                for line in response.iter_lines():
                    if line:
                        line_str = line.decode('utf-8')
                        if line_str.startswith('data: '):
                            data_str = line_str[6:]
                            if data_str.strip() == '[DONE]':
                                break
                            
                            try:
                                chunk = json.loads(data_str)
                                if "choices" in chunk and len(chunk["choices"]) > 0:
                                    delta = chunk["choices"][0].get("delta", {})
                                    content = delta.get("content", "")
                                    if content:
                                        print(content, end='', flush=True)
                                        full_response += content
                            except json.JSONDecodeError:
                                continue
                
                print(f"\n" + "-" * 40)
                print(f"✅ 流式输出完成，总长度: {len(full_response)} 字符")
                return True
            else:
                print(f"❌ 流式请求失败: {response.status_code}")
                return False
                
        except Exception as e:
            print(f"❌ 流式请求异常: {str(e)}")
            return False
    
    def test_generation(self, prompt="Once upon a time", max_tokens=50):
        """测试文本生成（非聊天格式）"""
        print(f"\n📝 测试文本生成")
        
        data = {
            "model": self.model,
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": 0.7,
            "stream": False
        }
        
        try:
            response = requests.post(
                f"{self.base_url}/completions",
                headers=self.headers,
                json=data,
                timeout=30,
                verify=self.verify_ssl
            )
            
            if response.status_code == 200:
                result = response.json()
                if "choices" in result and len(result["choices"]) > 0:
                    text = result["choices"][0].get("text", "")
                    print(f"✅ 文本生成成功:")
                    print("-" * 40)
                    print(prompt + text)
                    print("-" * 40)
                    return True
            else:
                print(f"❌ 文本生成失败: {response.status_code}")
                return False
                
        except Exception as e:
            print(f"❌ 文本生成异常: {str(e)}")
            return False
    
    def run_all_tests(self):
        """运行所有测试"""
        print("=" * 60)
        print("Ollama 本地模型测试")
        print("=" * 60)
        
        # 1. 检查Ollama服务
        print("\n1. 🚀 检查Ollama服务状态...")
        if not self.check_ollama_status():
            print("❌ 测试终止: Ollama服务未运行")
            return False
        
        # 2. 测试OpenAI兼容API
        print("\n2. 🔌 测试OpenAI兼容API...")
        if not self.test_openai_compatible_api():
            print("⚠️  OpenAI兼容API不可用，尝试直接测试模型...")
        
        # 3. 测试简单对话
        print("\n3. 💬 测试简单对话...")
        self.test_chat_completion(
            prompt="用中文介绍一下你自己",
            max_tokens=150
        )
        
        # 4. 测试推理能力
        print("\n4. 🧮 测试推理能力...")
        self.test_chat_completion(
            prompt="如果我有3个苹果，吃掉1个，又买了5个，现在有多少个苹果？",
            max_tokens=100
        )
        
        # 5. 测试流式输出
        print("\n5. 🌊 测试流式输出...")
        self.test_streaming(
            prompt="用三句话描述夏天的美好",
            max_tokens=80
        )
        
        # 6. 测试文本生成
        print("\n6. 📝 测试文本生成接口...")
        self.test_generation(
            prompt="人工智能的未来是",
            max_tokens=50
        )
        
        print("\n" + "=" * 60)
        print("✅ 所有测试完成!")
        print("=" * 60)
        return True

def main():
    # 配置
    config = {
        "base_url": "http://localhost:11434/v1",
        "model": "qwen3:0.6b ",
        "api_key": "ollama",
        "max_retries": 3,
        "verify_ssl": False
    }
    
    # 创建测试器
    tester = OllamaTester(config)
    
    # 运行测试
    try:
        tester.run_all_tests()
    except KeyboardInterrupt:
        print("\n\n⚠️  测试被用户中断")
    except Exception as e:
        print(f"\n❌ 测试过程中出现错误: {str(e)}")

if __name__ == "__main__":
    # 检查是否安装了requests
    try:
        import requests
    except ImportError:
        print("❌ 需要安装requests库")
        print("   请运行: pip install requests")
        sys.exit(1)
    
    main()

Ollama 本地模型测试

1. 🚀 检查Ollama服务状态...
✅ Ollama服务运行正常
   可用模型: qwen3:0.6b, dengcao/bce-reranker-base_v1:latest, deepseek-r1:8b, llama3.1:latest, gemma3:12b, qwen3:14b, bge-m3:latest, nomic-embed-text:latest, deepseek-r1:14b

2. 🔌 测试OpenAI兼容API...

🔍 测试OpenAI兼容端点: http://localhost:11434/v1
✅ OpenAI兼容API可用
   兼容端点检测到模型数量: 9
   ✅ 目标模型 'qwen3:0.6b' 可用

3. 💬 测试简单对话...

💬 测试聊天补全 (模型: qwen3:0.6b)
   尝试 1/3...
✅ 请求成功! 耗时: 1.56秒

📝 模型回复:
----------------------------------------
<think>
好的，用户让我介绍一下自己。首先，我需要确定用户的需求是什么。可能他想了解我的基本情况，或者想进行某种对话，比如询问我的兴趣、能力，甚至想进行某种互动。我需要根据这些可能性来组织回答。

用户可能没有明确说，但作为AI，我需要保持友好和专业的形象。要避免过于冗长，但又要涵盖关键信息。可能需要包括我的主要功能、应用场景、价值观，以及如何帮助用户。

另外，用户可能希望回答中包含一些互动元素，比如询问问题或者提供帮助。这时候可以主动询问，比如“有什么可以帮助你的吗？”。这不仅显示了开放的态度，也能促进进一步的交流。

需要注意的是，保持回答的
----------------------------------------

📊 Token使用:
   输入token: 12
   输出token: 150
   总token: 162

4. 🧮 测试推理能力...

💬 测试聊天补全 (模型: qwen3:0.6b)
   尝试 1/3...
✅ 请求成功! 耗时: 0.40秒

📝 模型回复:
----------------------------------------
<think>
好的，我现在要解决这个问题：如果我

In [1]:
print("\n\n测试脚本执行完毕。")



测试脚本执行完毕。
